## Ingest constructors.json file

1. Read the JSON file using the spark dataframe reader

In [0]:
from pyspark.sql.functions import col, lit

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
file_date= dbutils.widgets.get("p_file_date")

In [0]:
constructor_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
constructor_df = spark.read \
  .schema(constructor_schema) \
  .json(f'{raw_folder_path}/{file_date}/constructors.json')
#display(constructor_df)

2. Drop unnecessary columns

In [0]:
constructor_dropped_df = constructor_df.drop('url')

3. Rename columns and add ingestion date

In [0]:
constructor_final_df = constructor_dropped_df.withColumnRenamed("constructorId", "constructor_id") \
    .withColumnRenamed("constructorRef", "constructor_ref") \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(file_date))
contructor_final_df = add_ingestion_date(constructor_final_df)

4. Write data to datalake as parquet

In [0]:
#constructor_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/constructors")
# database
#constructor_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.constructors")
constructor_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
# %fs
# ls mnt/formula1jf/processed/constructors

In [0]:
dbutils.notebook.exit("Success")